<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://files.oproject.org/tmvalogo.png" height="50%" width="50%">

# TMVA  Reader Example 

#### Example of applying the trained classified of TMVA on a data set and evaluate the classifier

In [ ]:
import ROOT
from ROOT import TMVA

In [ ]:
##%jsroot on

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

### Choose TMVA method to evaluate

In [ ]:
methodNames = ["BDT","DL_CPU","Keras_Dense","PyGTB"]

###  Give input file 


In [ ]:
inputFile = ROOT.TFile("Higgs_data.root")

In [ ]:
inputFile.ls()
#inputFile.sig_tree.Print()

### Declare Reader


In [ ]:
TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize();

reader = TMVA.Reader( "!Color:!Silent" )


### Add Variable

We add variables to the reader exactly in the same way we did for the **DataLoader** during the training

We need to specify the address of the variable in order to pass it to TMVA when we iterate on the TTree


In [ ]:
from array import array
m_jj = array('f',[0])
m_jjj = array('f',[0])
m_lv = array('f',[0])
m_jlv = array('f',[0])
m_bb = array('f',[0])
m_wbb = array('f',[0])
m_wwbb = array('f',[0])

In [ ]:
#add variables 
reader.AddVariable("m_jj",m_jj)
reader.AddVariable("m_jjj",m_jjj)
reader.AddVariable("m_lv",m_lv)
reader.AddVariable("m_jlv",m_jlv)
reader.AddVariable("m_bb",m_bb)
reader.AddVariable("m_wbb",m_wbb)
reader.AddVariable("m_wwbb",m_wwbb)


## Setup Classifiers

We set up the classifiers by reading the input weights from the appropriate files

The file is stored for example as 

####  *dataset/weights/TMVAClassification_BDT.weights.xml

In [ ]:
for methodName in methodNames: 
    weightfile = "dataset/weights/TMVA_Higgs_Classification_" + methodName + ".weights.xml"
    name = ROOT.TString(methodName)
    reader.BookMVA( name, weightfile );

### Book histogram to visualize the response

In [ ]:
hs = {}
hb = {}
for name in methodNames:
    hname = "hsig_" + name
    htitle = "Classifier Output for " + name 
    hs[name] = ( ROOT.TH1D(hname,htitle,100,-1,1) )
    hname = "hbkg_" + name
    hb[name] = (ROOT.TH1D(hname,htitle,100,-1,1) )

## Iterate on the events from the input TTree 

### Evaluation of the classifier on the given input events

We iterate on the input event in the given TTree. We provide as input first the background tree 
We return the output results for the various methods in big numpy array [ number of methods x number of events]
We also fill an histogram for each method.
Note that is important to fill the arrays with the tree entries in order to pass the values to the TMVA::Reader

In [ ]:
import numpy as np

In [ ]:

nevt = inputFile.bkg_tree.GetEntries()

vout = np.arange( nevt * len(methodNames), dtype='float').reshape( len(methodNames), nevt)
ievt = 0
for entry in inputFile.bkg_tree:
    m_jj[0] = entry.m_jj
    m_jjj[0] = entry.m_jjj
    m_lv[0] = entry.m_lv
    m_jlv[0] = entry.m_jlv
    m_bb[0] = entry.m_bb
    m_wbb[0] = entry.m_wbb
    m_wwbb[0] = entry.m_wwbb
    
    i = 0
    for methodName in methodNames : 
        vout[i,ievt] = reader.EvaluateMVA(methodName)
        hb[methodName].Fill(vout[i,ievt])        
        #if (ievt%1000)==0 : print 'Event ',ievt,' MVA output for ',methodName,' =',vout[i,ievt]
        i += 1
        
    ievt += 1
#    if (ievt > 20000) : break
    

In [ ]:
print(vout)
bkg_result = vout

### Evaluation on the signal  events

We evaluate now the classifier on the another tree (e.g. the signal events)



In [ ]:
nevt = inputFile.sig_tree.GetEntries()

vout = np.arange( nevt * len(methodNames), dtype='float').reshape( len(methodNames), nevt)
ievt = 0
for entry in inputFile.sig_tree:
    m_jj[0] = entry.m_jj
    m_jjj[0] = entry.m_jjj
    m_lv[0] = entry.m_lv
    m_jlv[0] = entry.m_jlv
    m_bb[0] = entry.m_bb
    m_wbb[0] = entry.m_wbb
    m_wwbb[0] = entry.m_wwbb
    
    i = 0
    for methodName in methodNames : 
        vout[i,ievt] = reader.EvaluateMVA(methodName)
        hs[methodName].Fill(vout[i,ievt])
        #if (ievt%1000)==0 : print 'Event ',ievt,' MVA output for ',methodName,' =',vout[i,ievt]
        i += 1
        
    ievt += 1
#    if (ievt > 20000) : break
    
    
sig_result = vout
print(sig_result)

### Plot Histogram of output variables

In [ ]:
c1 = ROOT.TCanvas()
c1.DivideSquare( len(hb))
ipad = 0
for name in methodNames: 
    ipad += 1
    pad = c1.cd(ipad)
    h1 = hb[name]
    h1.Draw()
    h2 = hs[name]
    h2.SetLineColor(ROOT.kRed)
    h2.Draw('Same')
    pad.BuildLegend()
    
c1.Draw()

### Computing ROC Curve

We compute here the ROC curve from the vector of the classifier output for signal and background events. 
The interface for the TMVA::ROCCurve class requires  a std::vector of float types

In [ ]:
i = 0
roc = []
c2 = ROOT.TCanvas()
for name in methodNames: 
    vsig = ROOT.std.vector('float')(nevt)
    vbkg = ROOT.std.vector('float')(nevt)
    for ievt in range(0,nevt): 
        vsig[ievt] = sig_result[i,ievt]
        vbkg[ievt] = bkg_result[i,ievt]
    roc.append ( TMVA.ROCCurve(vsig,vbkg) )
    print name,' ROC Integral ',roc[i].GetROCIntegral()
    gr = roc[i].GetROCCurve()
    gr.SetLineColor(i+1)
    gr.SetTitle(name)
    if (i==0): 
        gr.Draw('AL')
    else:
        gr.Draw("L")
    i += 1
    


In [ ]:
c2.BuildLegend()
c2.Draw()